| Concept           | Frequency Range   | Notes / MIDI Range      | Notes                        |
| ----------------- | ----------------- | ----------------------- | ---------------------------- |
| Audible (human)   | 20 Hz – 20 kHz    | MIDI 0 – 127            | Full MIDI                    |
| Playful / Musical | 27.5 Hz – 4186 Hz | MIDI 21 (A0) – 108 (C8) | Piano & general instruments  |
| Safe / Harmonics  | 80 Hz – 12 kHz    | MIDI ~28 – 110          | Most instruments and effects |


### MIDI Note Numbers to Frequency

The formula to convert a MIDI note number to frequency (Hz) is:

$$
f = 440 \times 2^{\frac{(n - 69)}{12}}
$$

Where:
- \( f \) = frequency in Hz
- \( n \) = MIDI note number
- 440 Hz = standard tuning for A4 (MIDI note 69)

**Example:**
- Middle C (MIDI 60):
$$
f = 440 \times 2^{\frac{60 - 69}{12}} \approx 261.63 \text{ Hz}
$$

---

### Frequency to MIDI Note Number

To find the MIDI note number from a frequency:

$$
n = 69 + 12 \times \log_2\left(\frac{f}{440}\right)
$$

**Example:**
- Frequency 523.25 Hz (C5):
$$
n = 69 + 12 \times \log_2\left(\frac{523.25}{440}\right) \approx 72
$$

---

### MIDI Note Number to Musical Note

A common mapping for note names:

```text
0 = C-1
1 = C#-1 / Db-1
2 = D-1
...
60 = C4 (Middle C)
69 = A4


In [17]:
# List of note names in an octave
note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

print(f"{'MIDI':<5} {'Note':<4}")
print('-' * 10)

# MIDI note numbers typically range from 0 to 127
for midi_num in range(69,128):
    octave = (midi_num // 12) - 1      # MIDI standard: C4 = 60
    note = note_names[midi_num % 12]
    print(f"{midi_num:<5} {note}{octave:<3}")


MIDI  Note
----------
69    A4  
70    A#4  
71    B4  
72    C5  
73    C#5  
74    D5  
75    D#5  
76    E5  
77    F5  
78    F#5  
79    G5  
80    G#5  
81    A5  
82    A#5  
83    B5  
84    C6  
85    C#6  
86    D6  
87    D#6  
88    E6  
89    F6  
90    F#6  
91    G6  
92    G#6  
93    A6  
94    A#6  
95    B6  
96    C7  
97    C#7  
98    D7  
99    D#7  
100   E7  
101   F7  
102   F#7  
103   G7  
104   G#7  
105   A7  
106   A#7  
107   B7  
108   C8  
109   C#8  
110   D8  
111   D#8  
112   E8  
113   F8  
114   F#8  
115   G8  
116   G#8  
117   A8  
118   A#8  
119   B8  
120   C9  
121   C#9  
122   D9  
123   D#9  
124   E9  
125   F9  
126   F#9  
127   G9  


In [18]:
!pip install mido


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [19]:
from mido import Message, MidiFile, MidiTrack


## 1. Beats

In [20]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

KICK = 45; SNARE = 48; HIHAT = 63
ticks = 300


pattern = [
    (KICK, 0),
    (HIHAT, 0),
    (SNARE, ticks),
    (HIHAT, 0),
    (KICK, ticks),
    (HIHAT, 0),
    (SNARE, ticks),
    (HIHAT, 0)
]

for note, delay in pattern:
    track.append(Message('note_on', note=note, velocity=100, time=delay))
    track.append(Message('note_off', note=note, velocity=100, time=120))

mid.save("midi-clips/beat.mid")


## 2. Harmony Clip (Chord Progression)

In [21]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

chords = [
    [60,64,67],   # C
    [67,71,74],   # G
    [69,72,76],   # Am
    [65,69,72],   # F
]

for chord in chords:
    for n in chord:
        track.append(Message('note_on', note=n, velocity=80, time=0))
    for n in chord:
        track.append(Message('note_off', note=n, velocity=80, time=960))

mid.save("midi-clips/harmony.mid")


## 3. Melody Clip (Long Single Clip)

In [22]:
mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

melody_notes = [60, 62, 64, 67, 69, 67, 64, 62] * 3  # 16-bar idea
time_step = 250

for note in melody_notes:
    track.append(Message('note_on', note=note, velocity=90, time=time_step))
    track.append(Message('note_off', note=note, velocity=90, time=240))

mid.save("midi-clips/melody.mid")


## 4. Play MIDI with pygame

In [23]:

!pip install pygame
!pip install pygame

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [24]:
import pygame

pygame.mixer.init()
pygame.mixer.music.load("midi-clips/melody.mid")
pygame.mixer.music.play()

print("Playing MIDI...")
while pygame.mixer.music.get_busy():
    pass


Playing MIDI...


In [25]:
!pip install fluidsynth


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## 5. MIDI to Audio 

In [26]:
import os
import subprocess

sf = os.path.expanduser("~/Downloads/FluidR3GM2-2.SF2")

clips = ["beat", "harmony", "melody"]

for clip in clips:
    midi = "midi-clips/"+clip+".mid"
    out = "audio-clips/"+clip+".wav"
    Fluidsynth_path=r"C:\Users\ACER\Downloads\fluidsynth-v2.5.2-win10-x64-cpp11\fluidsynth-v2.5.2-win10-x64-cpp11\bin\fluidsynth.exe"
    cmd = [
    Fluidsynth_path,
    "-n",             # no audio driver
    "-i",             # no shell
    "-F", out,        # write to file
    "-r", "44100",    # sample rate
    sf,               # SoundFont
    midi              # MIDI file
    ]

    print("Running:", " ".join(cmd))
    subprocess.run(cmd)
    print("Saved:", out)


Running: C:\Users\ACER\Downloads\fluidsynth-v2.5.2-win10-x64-cpp11\fluidsynth-v2.5.2-win10-x64-cpp11\bin\fluidsynth.exe -n -i -F audio-clips/beat.wav -r 44100 C:\Users\ACER/Downloads/FluidR3GM2-2.SF2 midi-clips/beat.mid
Saved: audio-clips/beat.wav
Running: C:\Users\ACER\Downloads\fluidsynth-v2.5.2-win10-x64-cpp11\fluidsynth-v2.5.2-win10-x64-cpp11\bin\fluidsynth.exe -n -i -F audio-clips/harmony.wav -r 44100 C:\Users\ACER/Downloads/FluidR3GM2-2.SF2 midi-clips/harmony.mid
Saved: audio-clips/harmony.wav
Running: C:\Users\ACER\Downloads\fluidsynth-v2.5.2-win10-x64-cpp11\fluidsynth-v2.5.2-win10-x64-cpp11\bin\fluidsynth.exe -n -i -F audio-clips/melody.wav -r 44100 C:\Users\ACER/Downloads/FluidR3GM2-2.SF2 midi-clips/melody.mid
Saved: audio-clips/melody.wav


## PyDub

In [27]:
!pip install pydub

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [28]:
from pydub import AudioSegment
from pydub.playback import play

# Load pre-rendered clips
beat = AudioSegment.from_wav("audio-clips/beat.wav")          # short loop
harmony = AudioSegment.from_wav("audio-clips/harmony.wav")    # short loop
melody = AudioSegment.from_wav("audio-clips/melody.wav")      # long single clip

# Repeat beat & harmony to match melody length
repeat_count = int(len(melody) / len(beat)) + 1
beat_loop = beat * repeat_count
harmony_loop = harmony * repeat_count

# Trim loops to melody length
beat_loop = beat_loop[:len(melody)]
harmony_loop = harmony_loop[:len(melody)]

# Mix all tracks
final_mix = beat_loop.overlay(harmony_loop).overlay(melody)

# Play
# play(final_mix)

# export
final_mix.export("audio-clips/final_song_pydub.wav", format="wav")


<_io.BufferedRandom name='audio-clips/final_song_pydub.wav'>

In [29]:
!pip install midi2audio
! pip install pygame
! pip install librosa

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Librosa

In [30]:
import librosa
import soundfile as sf
import numpy as np
from IPython.display import Audio

# Load audio as numpy arrays
beat, sr = librosa.load("audio-clips/beat.wav", sr=None)
harmony, _ = librosa.load("audio-clips/harmony.wav", sr=None)
melody, _ = librosa.load("audio-clips/melody.wav", sr=None)

# Repeat beat & harmony to match melody length
repeat_count = int(len(melody) / len(beat)) + 1
beat_loop = np.tile(beat, repeat_count)[:len(melody)]
harmony_loop = np.tile(harmony, repeat_count)[:len(melody)]

# Mix tracks
final_mix = beat_loop + harmony_loop + melody

# Normalize to prevent clipping
final_mix = final_mix / np.max(np.abs(final_mix))

# Save final audio
sf.write("audio-clips/final_song_librosa.wav", final_mix, sr)

# Play in notebook
Audio(final_mix, rate=sr)


In [31]:
! pip install sounddevice

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\ACER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [32]:
import sounddevice as sd
sd.play(final_mix, sr)
print(final_mix.shape)

sd.wait()


(628736,)
